In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

# Model

In [ ]:
from src.common.utils import load_model
from src.models.resnets_eff import ResNet18

NUM_CLASSES = 10
model = ResNet18(num_classes=NUM_CLASSES).to(device)

PATH = 'models/resnet18_100_epochs.pth'
model = load_model(model, PATH)

for name, p in model.named_parameters():
    p.requires_grad = False

# Early Exit Wrapper

In [ ]:
from src.models.methods.sdn import SDN
LD = [64, 64, 128, 128, 256, 256, 512, 512]

config_ee = {
    'confidence_metric': 'entropy',
    'confidence_threshold': 0.05,
    'is_model_frozen': True,
    'act_name': 'relu',
    'dropout_prob': 0.1,
    'reduction_layer_weight_std': 0.02,
    'device': device
}

ee_wrapper = SDN(model=model, layers_dim=LD, num_classes=NUM_CLASSES, config_ee=config_ee).to(device)

# Dataset

In [ ]:
from torch.utils.data import DataLoader
from src.data.datasets import get_cifar10


train_dataset, _, test_dataset = get_cifar10('data/')

BATCH_SIZE = 64

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=4)

loaders = {
    'train': train_loader,
    'test': test_loader
}

# Optimizer

In [ ]:
from src.common.common import OPTIMIZER_NAME_MAP, LOSS_NAME_MAP, SCHEDULER_NAME_MAP

GRAD_ACCUM_STEPS = 1
EPOCHS = 20
T_max = (len(train_loader) // GRAD_ACCUM_STEPS) * EPOCHS

optim = OPTIMIZER_NAME_MAP['adamw'](filter(lambda p: p.requires_grad, ee_wrapper.parameters()), **{'lr': 0.01, 'weight_decay': 0.001})
lr_scheduler = SCHEDULER_NAME_MAP['cosine'](optim, T_max=T_max, eta_min=1e-6)
T_max

# Tensorboard

In [ ]:
%tensorboard --logdir=reports/sdn_resnet18_pretrained_100_epochs

# Trainer

In [ ]:
from src.trainer.trainer_ee import EarlyExitTrainer

params_trainer = {
    'ee_wrapper': ee_wrapper,
    'loaders': loaders,
    'optim': optim,
    'lr_scheduler': lr_scheduler,
    'accelerator': None
}

trainer = EarlyExitTrainer(**params_trainer)

In [ ]:
from src.common.utils import AttrDict

EXP_NAME = 'sdn_resnet18_pretrained_100_epochs'

config = {
    'epoch_start_at': 0,
    'epoch_end_at': EPOCHS,
    'grad_accum_steps': GRAD_ACCUM_STEPS,
    'save_multi': T_max // 10,
    'log_multi': 100,
    'whether_clip': False,
    'clip_value': 2.0,
    'base_path': 'reports',
    'exp_name': EXP_NAME,
    'logger_name': 'tensorboard',
    'logger_config': {'api_token': "07a2cd842a6d792d578f8e6c0978efeb8dcf7638", 'project': 'early_exit', 'hyperparameters': {}},
    'random_seed': 42,
    'device': device

}
config = AttrDict(config)

trainer.run_exp(config)